# Golly Sportsbook

## SQL Table Setup and Data Classes

This notebook sets up some data classes for tables useful for setting up a Golly sports book.

In [14]:
import os, sys, subprocess, json, time

In [15]:
from sqlalchemy import Column, Integer, String, Boolean, Table
from sqlalchemy.orm import relationship, backref
from sqlalchemy.orm import DeclarativeBase

## Game Result Table

This table stores a simplified version of each game and its outcome. This table is used to compute other odds.

In [16]:
class Base(DeclarativeBase):
    pass

In [17]:
class GameResult(Base):
    __tablename__ = "sportsbook_game_result"
    gameid = Column("gameid", String, primary_key=True)
    cup = Column("cup", String)
    season = Column("season", Integer)
    day = Column("day", Integer)
    mapName = Column("mapName", String)
    winnerAbbr = Column("winnerAbbr", String)
    winnerPoints = Column("winnerPoints", Integer)
    loserAbbr = Column("loserAbbr", String)
    loserPoints = Column("loserPoints", Integer)
    isPostseason = Column("isPostseason", Boolean)
    series = Column("series", String)
    generations = Column("generations", Integer)

## Moneyline Odds Table

This table stores the moneyline odds for each game, so that they only need to be computed once.

In [18]:
class Moneyline(Base):
    __tablename__ = "sportsbook_moneyline"
    gameid = Column("gameid", String, primary_key=True)
    cup = Column("cup", String)
    season = Column("season", Integer)
    day = Column("day", Integer)
    mapName = Column("mapName", String)
    team1Abbr = Column("team1Abbr", String)
    team1Odds = Column("team1Odds", Integer)
    team2Abbr = Column("team2Abbr", String)
    team2Odds = Column("team2Odds", Integer)

## Map Generations Table and Map Points Table

The map generations table stores the median number of generations reached on each map, for use in generation over/under bets.

The map points table stores the number of points scored on each map (by total and by team). These numbers are used to compute spreads and total points over/under bets. They are not used directly.

In [19]:
class MapGenerations(Base):
    __tablename__ = "sportsbook_map_generations"
    id = Column("id", Integer, primary_key=True)
    cup = Column("cup", String)
    mapName = Column("mapName", String)
    # This is the median number of generations scored on this map
    generations = Column("generations", Integer)

In [20]:
class MapPoints(Base):
    __tablename__ = "sportsbook_map_points"
    id = Column("id", Integer, primary_key=True)
    cup = Column("cup", String)
    season = Column("season", Integer)
    day = Column("day", Integer)
    mapName = Column("mapName", String)
    # If teamAbbr is empty, it means this is the median for all games on this map
    teamAbbr = Column("teamAbbr", String)
    # This is the median number of points scored by this team on this map
    teamPoints = Column("teamPoints", Integer)

## Spread Bets Table

This table stores information about the point spread for each game. This is storing the final computed spread for spread bets.

In [21]:
class Spreads(Base):
    __tablename__ = "sportsbook_spreads"
    gameid = Column("gameid", String, primary_key=True)
    cup = Column("cup", String)
    season = Column("season", Integer)
    day = Column("day", Integer)
    mapName = Column("mapName", String)
    # The overdog will get the - number
    overdogName = Column("overdogName", String)
    # The underdog will get the + number
    underdogName = Column("underdogName", String)
    spread = Column("spread", Integer)

## Create Tables

Now we'll create each table...

In [22]:
from sqlalchemy.schema import CreateTable
from sqlalchemy.dialects import sqlite

In [23]:
#print(CreateTable(GameResult.__table__).compile(dialect=sqlite.dialect()))

In [24]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

fpath = os.path.join(os.environ['HOME'], 'tmp', 'sqlite', 'mydb.db')
engine = create_engine(f"sqlite:///{fpath}")
Session = sessionmaker(engine)

In [25]:
# Start by creating all the tables
Base.metadata.create_all(engine)

In [26]:
# Run a simple query on the tables
with Session() as session:
    results = session.query(GameResult).all()

print(len(results))

23748
